In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [6]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [7]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
# sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [8]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [15]:
edm_dir = r'/data/new_trap/20210622/bead1/edm_measurement/'
voltage = [55, 65, 75, 85, 95]
fname = 'edm_spin_46700_'+str(voltage)+'V.h5'
fnames = ['edm_spin_46700_'+str(voltage_)+'V.h5' for voltage_ in voltage]
edm_files = [BeadDataFile.BeadDataFile(edm_dir+fname_) for fname_ in fnames] 

No height data
No electrodes data
No cantilever data
No laser power data
No height data
No electrodes data
No cantilever data
No laser power data
No height data
No electrodes data
No cantilever data
No laser power data
No height data
No electrodes data
No cantilever data
No laser power data
No height data
No electrodes data
No cantilever data
No laser power data


In [80]:
fsamp = 5000
res = 5000
factor = 40
freq_side_band = []
for bb in edm_files:
    spsd1, freqs1_s = matplotlib.mlab.psd(bb.spin_data[0], Fs = fsamp*factor, NFFT = res*factor)
    ind_max = np.argmax(spsd1[20000:]) 
    ind_max2 = np.argmax(spsd1[20050+ind_max:20050+ind_max+50])
    freq_side_band.append(freqs1_s[ind_max2+20050+ind_max]-freqs1_s[ind_max+20000])

In [81]:
_ = [print(freq_, voltage_) for freq_,voltage_ in zip(freq_side_band, voltage)]

51.0 55
54.0 65
64.0 75
63.0 85
71.0 95


In [82]:
efiled = 2*np.array(voltage)/8e-4*0.61 #V/m
I = 2/5*420e-15*(7.6e-6/2)**2 #kg m2
omega_squared = (np.array(freq_side_band)*2*np.pi)**2 #rad/sec
d = omega_squared*I/efiled
d/1.6e-19/1e-6, np.mean(d/1.6e-19/1e-6), np.std(d/1.6e-19/1e-6) ## converted to e um

(array([18.56196921, 17.60842648, 21.43606523, 18.32772495, 20.82761195]),
 19.352359563136808,
 1.498939258992453)

In [89]:
_,ax = plt.subplots()
ax.scatter(voltage, d/1.6e-19/1e-6)
ax.set(ylim=(10,30), ylabel=r'EDM [$e \mu m$]', xlabel='Rotation amplitude [V]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(10.0, 30.0),
 Text(0, 0.5, 'EDM [$e \\mu m$]'),
 Text(0.5, 0, 'Rotation amplitude [V]')]

In [90]:
BDF = edm_files[-1]
fsamp = 5000
res = 5000
factor = 40
spsd1, freqs1_s = matplotlib.mlab.psd(BDF.spin_data[0], Fs = fsamp*factor, NFFT = res*factor)

_,ax = plt.subplots()
ax.semilogy(freqs1_s, np.sqrt(spsd1), '.-',markersize=2,linewidth=1)
# ax.legend()
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('ASD [a.u.]')
ax.set_title('spin')
# ax.set(xlim=(46000,47500), ylim=(1e-7,1e-3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'spin')